# Visemenet Inference Demo
- Visemenet: Lip Snycronization from speech LSTM Model (https://arxiv.org/abs/1805.09488)

In [34]:
!pip install tensorflow
!pip install python_speech_features
!pip install scipy
!pip -q install pydub
!git clone https://github.com/junhwanjang/visemenet-inference.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'visemenet-inference' already exists and is not an empty directory.


In [35]:
import sys
sys.path.append("./visemenet-inference/")


In [ ]:
!ffmpeg -i /content/visemenet-inference/test_audio/obama_speech.wav -ar 44100 /content/visemenet-inference/test_audio/obama_speech-enc.wav

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

In [71]:
import pydub
# m4a_file_path = "./visemenet-inference/test_audio/chinese_test.m4a"
wav_file_path = "./visemenet-inference/test_audio/obama_speech-enc.wav"

# track = pydub.AudioSegment.from_file(m4a_file_path,  format= 'm4a')
# file_handle = track.export(wav_file_path, format='wav')


In [72]:
# wav_file_path = "./visemenet-inference/test_audio/visemenet_intro.wav"
res = pydub.AudioSegment.from_file(wav_file_path)
print(res.duration_seconds)
pydub.AudioSegment.from_file(wav_file_path)

20.01560090702948


### Doing Inference to Custom Audio Input

In [78]:
from inference import VisemeRegressor

viseme_regressor = VisemeRegressor("./visemenet-inference/visemenet_frozen.pb")

mean_std_csv_path = "./visemenet-inference/saved_params/wav_mean_std.csv"
close_face_txt_path = "./visemenet-inference/saved_params/maya_close_face.txt"

viseme_outputs = viseme_regressor.predict_outputs(
    wav_file_path=wav_file_path,
    mean_std_csv_path=mean_std_csv_path, close_face_txt_path=close_face_txt_path
)


input/Placeholder_1
input/Placeholder_2
input/phase
net1_shared_rnn/Placeholder


In [79]:
# print(viseme_outputs)
schwa = viseme_outputs[...,11]
schwa[schwa<0.1]=0
# schwa[schwa<0.1]=schwa[schwa<0.1]
# viseme_outputs[...,11] = schwa
schwa[(0.1<=schwa) & (schwa<0.2)] = (schwa[(0.1<=schwa) & (schwa<0.2)] - 0.1)*2
viseme_outputs[...,11] = schwa
viseme_outputs = np.append(viseme_outputs, viseme_outputs[:,-1:], axis=1)
# viseme_outputs[viseme_outputs<0.1]=0
# viseme_outputs[(0.1<=viseme_outputs) & (viseme_outputs<0.2)] = (viseme_outputs[(0.1<=viseme_outputs) & (viseme_outputs<0.2)] - 0.1)*2
# viseme_outputs = np.append(viseme_outputs, viseme_outputs[:,-1:], axis=1)

In [80]:
import numpy as np
curr_frame = 40
frame_length = np.shape(viseme_outputs)[0]
vis_eye = np.zeros(frame_length)
close_eye = np.array([0+0.2*i for i in range(5)])
input_frame = np.concatenate((close_eye,np.array([1]), close_eye[::-1]))
while(curr_frame<frame_length-10):
  vis_eye[curr_frame-6:curr_frame+5] = input_frame
  # vis_eye[curr_frame] = 1
  curr_frame += 120
viseme_outputs[...,22] = vis_eye

[0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.2
 0.4 0.6 0.8 1.  0.8 0.6 0.4 0.2 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.2 0.4 0.6 0.8 1.  0.8 0.6
 0.4 0.2 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

In [81]:
import numpy as np
additional_frame = 3
last_frame = viseme_outputs[-1]

for i in range(additional_frame):
  f = last_frame*((additional_frame-i-1)/additional_frame)
  f[0] = last_frame[0]
  viseme_outputs = np.concatenate((viseme_outputs, [f]), axis=0)

# viseme_outputs[viseme_outputs<0.01]=0


In [82]:
import numpy as np

out_txt_path = "./maya_viseme_outputs.txt"
np.savetxt(out_txt_path, viseme_outputs, "%.4f")

### Generating Json Format Output

In [53]:
import json
output = []
out_json_path = "./output_json.json"
param_public_model = ['JAW', 'Lip', 'Aaa', 'Eh', 'Ahh', 'Ohh', 'Uuu', 'Lee', 'Rrr', 'Www', 'Sss', 'Fff', 'Tth', 'Mbp', 'Ssh', 'Schwa', 'Gk', 'Lntd', 'Lntd1', 'GK1', 'MMM', 'FFF1', 'WaPedal', 'YanJing_21']
map_i = [0, 1, 3, 4, 2, [7, 8], 9, [5, 6], 12, 9, 13, 20, 15, 19, 14, 11, 18, 17, 17, 18, 19, 20, 21, 22]
print(len(param_public_model), len(map_i))
for line in viseme_outputs:
  dict_i = {}
  for i in range(len(param_public_model)):
    if(type(map_i[i]) == list):
      tmp = min(max(line[map_i[i][0]], line[map_i[i][1]], 0), 1)
    else:
      tmp = min(max(line[map_i[i]], 0), 1)

    if i in [1, 22]:
      dict_i[param_public_model[i]] = tmp*1
    elif i in [0]:
      dict_i[param_public_model[i]] = tmp*0
    elif i in [5,8,10,11,13,14,16,17]:
      dict_i[param_public_model[i]] = tmp*0.8
    elif i in [2,3,4,6,7,9]:
      dict_i[param_public_model[i]] = tmp*0.8
    elif i in [15]:
      dict_i[param_public_model[i]] = tmp*0.7
    else:   
      dict_i[param_public_model[i]] = tmp*0.8
  output.append(dict_i) 
output_dict = {"list": output}

with open(out_json_path, "w") as outfile:
    json.dump(output_dict, outfile)


24 24


In [ ]:
f = open("/content/output_json.json",)
js = json.load(f)
len(js["list"])

217